# Analyzing Data From WCA Competitions

In this project I use python to analyze data on Rubik's Cube competitions hosted by the WCA, or "World Cube Association." 
Many different events are held besides the traditional 3x3 cube. There are a total of 17 events, including 3x3 blindfolded and 7x7. The competitions are structured in the following manner: There are a certain number of rounds for each event, typically 2-4 rounds. In each round, competitors will generally do 5 solves. The worst and best time are thrown out and the middle 3 are averaged. This determines the ranking for the competitors in each round. Less competitors advanced after each round, with the final round typically being 12-16 competitors. The person with the fastest average time in the final round wins the event for that competition. For more information on the WCA and WCA competitions, visit the WCA website [here](https://www.worldcubeassociation.org/). The dataset can be exported from [here](https://www.worldcubeassociation.org/export/results).

## Loading the Data

First we will import the necessary libraries.

In [32]:
import pandas as pd

Next we need to read the data into pandas dataframes. The data was exported from the WCA website in tsv format. The necessary data is contained within 3 seperate tsv files we need to read. Firstly, there is the "results" file. This file shows information on individual averages done in WCA competitions. Secondly, there is the "persons" file, which shows information on every WCA competitor. We will use this file only to get the gender of each competitor. Finally, there is the "competitions" file, which contains information on individual WCA competitions. 

In [18]:
results = pd.read_csv(r'Data/WCA_export_Results.tsv', sep='\t', usecols=["competitionId", "eventId", "roundTypeId", "personName", "personId", "value1", "value2", "value3", "value4", "value5", "personCountryId"])
print(results.head())

  competitionId eventId roundTypeId               personName    personId  \
0  LyonOpen2007     333           1            Etienne Amany  2007AMAN01   
1  LyonOpen2007     333           1           Thomas Rouault  2004ROUA01   
2  LyonOpen2007     333           1  Antoine Simon-Chautemps  2005SIMO01   
3  LyonOpen2007     333           1           Irène Mallordy  2007MALL01   
4  LyonOpen2007     333           1       Marlène Desmaisons  2007DESM01   

   value1  value2  value3  value4  value5 personCountryId  
0    1968    2203    2138    2139    2108   Cote d_Ivoire  
1    2222    2153    1731    2334    2046          France  
2    3430    2581    2540    2789    2305          France  
3    2715    2452    2868    2632    2564          France  
4    2921    3184    2891    2677    2907          France  


In [10]:
persons = pd.read_csv(r"Data/WCA_export_Persons.tsv", sep="\t", usecols=["id", "gender"])
print(persons.head())

  gender          id
0      m  1982BORS01
1      m  1982BRIN01
2      m  1982CHIL01
3      f  1982FRID01
4      f  1982FRID01


In [12]:
competitions = pd.read_csv(r"Data/WCA_export_Competitions.tsv", sep="\t", usecols=["id", "countryId", "year", "month", "day"])
print(competitions.head())

                             id  countryId  year  month  day
0                 100Merito2018     Brazil  2018      4   14
1    100YearsRepublicAnkara2023     Turkey  2023     10   28
2  100YearsRepublicIstanbul2023     Turkey  2023     10   28
3      100YilMBACubeWeekend2023     Turkey  2023     12   16
4    10AniversarioGuatemala2023  Guatemala  2023     10   14


## Data Cleaning and Wrangling

Next, we need to merge the datasets together into one dataframe.

In [29]:
data = results.merge(persons, left_on = "personId", right_on = "id")
data = data.merge(competitions, left_on = "competitionId", right_on = "id")
data = data.drop(columns=["id_x", "id_y", "competitionId"])
data= data.rename(columns={"countryId": "competitionCountryId"})
print(data.head())

  eventId roundTypeId               personName    personId  value1  value2  \
0     333           1            Etienne Amany  2007AMAN01    1968    2203   
1     333           1           Thomas Rouault  2004ROUA01    2222    2153   
2     333           1  Antoine Simon-Chautemps  2005SIMO01    3430    2581   
3     333           1           Irène Mallordy  2007MALL01    2715    2452   
4     333           1       Marlène Desmaisons  2007DESM01    2921    3184   

   value3  value4  value5 personCountryId gender competitionCountryId  year  \
0    2138    2139    2108   Cote d_Ivoire      m               France  2007   
1    1731    2334    2046          France      m               France  2007   
2    2540    2789    2305          France      m               France  2007   
3    2868    2632    2564          France      f               France  2007   
4    2891    2677    2907          France      f               France  2007   

   month  day  
0      9    1  
1      9    1  
2      9

Now we need to reformat the data a bit. I want every row in dataframe to be for one solve, so I need to combine the value columns into a single column.